In [ ]:
from google.colab import files
files.upload() # expire any previous token(s) and upload recreated token

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download andrewmvd/face-mask-detection
!unzip -q face-mask-detection.zip -d face-mask-detection # outputs supressed

In [ ]:
# New folder to hold processed labels for yolov3 format
! mkdir face-mask-detection/labels

In [ ]:
import xml.etree.ElementTree as ET
from os import listdir

def create_txt_labels():
    img_directory = '/content/face-mask-detection/images/'
    annotation_directory = '/content/face-mask-detection/annotations/'

    for image in listdir(img_directory):
      image_name = image[:-4]
      annotation_path = annotation_directory + image_name + '.xml'

      # parse the xml file
      tree = ET.parse(annotation_path)
      root = tree.getroot()

      to_write = []

      # Get the width and height
      for dim in root.iter('size'):
        width = float(dim[0].text)
        height = float(dim[1].text)

      for object in root.iter('object'):
        # Class
        name = object[0].text

        # Class definition
        # No Mask: 0
        # Mask: 1
        # Incorrect: 2
        if name == 'mask_weared_incorrect':
          obj_class = 2
        elif name == 'with_mask':
          obj_class = 1
        elif name == 'without_mask':
          obj_class = 0

        # Coordinates of the bounding box
        xmin = int(object[5][0].text)
        ymin = int(object[5][1].text)
        xmax = int(object[5][2].text)
        ymax = int(object[5][3].text)

        # Find midpoint of bounding box
        xmid = (xmin + xmax) / 2
        ymid = (ymin + ymax) / 2

        # Find width of bounding box
        box_width = xmax - xmin
        box_height = ymax - ymin

        # Normalize the values
        xmid = float(xmid) / width
        ymid = float(ymid) / height
        box_width = float(box_width) / width
        box_height = float(box_height) / height

        # Store
        to_write.append("{} {} {} {} {}".format(obj_class, xmid, ymid, box_width, box_height))

      # Make path to file
      label_path = '/content/face-mask-detection/labels/' + image_name + '.txt'

      # Create and write to file
      print("\n".join(to_write), file= open(label_path, "w"))
